In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
import lightgbm as lgb
import polyline
import tqdm

In [2]:
print('pandas: version {}'.format(pd.__version__))
print('sklearn: version {}'.format(sklearn.__version__))
print('numpy: version {}'.format(np.__version__))
print('tqdm: version {}'.format(tqdm.__version__))
print('lgb: version {}'.format(lgb.__version__))
print('polyline: version {}'.format(polyline.__version__))

# pandas: version 0.25.3
# sklearn: version 0.21.2
# numpy: version 1.16.2
# tqdm: version 4.38.0
# lgb: version 2.3.0
# polyline: version 1.4.0

pandas: version 0.25.3
sklearn: version 0.21.2
numpy: version 1.16.2
tqdm: version 4.38.0
lgb: version 2.3.0
polyline: version 1.4.0


## Загрузка данных

In [83]:
dataset_place = 'test_additional.csv'
test = pd.read_csv(dataset_place)

## Генерация признаков

In [84]:
# Функция для подсчета дистанции между 2 точками
def dist(x_0, x_1):
    return ((x_0[0] - x_1[0]) ** 2 + (x_0[1] - x_1[1]) ** 2) ** (1/2)

In [85]:
# Импортируем необходимые для предсказаний переменный, которые мы получили при обучение модели
import pickle
with open('bst.pickle', 'rb') as f:
    bst, best_k = pickle.load(f)
    
with open('additional_data.pickle', 'rb') as f:
    global_mean, mean_dict, cnt_dict  = pickle.load( f)
    
train_cols = ['ETA', 'latitude', 'del_latitude', 'longitude', 'del_longitude', 'hour', 'weekday', 'e_speed', 'dist',
              'dist_center_start', 'div_dist', 'mean_speed_encoding', 'len_route', 'max_route_part']

In [98]:
def predict(test, bst, best_k, global_mean, mean_dict, cnt_dict):
    # Переводим время в нужный формат
    test['OrderedDate'] = pd.to_datetime(test['OrderedDate'])
    # Добавляем час и день недели
    test['hour'] = test['OrderedDate'].dt.hour
    test['weekday'] = test['OrderedDate'].dt.weekday
    # Считаем предпологаемую скорость
    test['e_speed'] = test['ETA'] / test['EDA']
    # Считаем дистанцию между началом и концом маршрута
    test['dist'] = dist([test['latitude'].values, test['longitude'].values], 
                [test['del_latitude'].values, test['del_longitude'].values])
    # Считаем дистанцию между началом маршрута и центром города
    test['dist_center_start'] = dist([test['latitude'].values, test['longitude'].values], 
                [test['center_latitude'].values, test['center_longitude'].values])
    # Смотрим отношения расстояния между началом и концом маршрута к предпологаемому расстоянию
    test['div_dist'] = test['dist'] / test['EDA']

    alpha = 20
    test['hour_locality'] = test['hour'].astype(str) + '_' + test['main_id_locality'].astype(str)
    # Считаем реальную среднюю скорость
    test['mean_speed_encoding'] = -1
    # сглаженная средняя скорость
    mean_target = []
    for x in test['hour_locality'].values:
        mean_target += [(alpha * global_mean + mean_dict.get(x, 0) * cnt_dict.get(x, 0)) / (alpha + cnt_dict.get(x, 0))]
    test['mean_speed_encoding'] = mean_target
    # Для предпологаемых путей посчитаем количество точек в маршруте
    route_list = []
    for route in test['route'].values:
        if route == route:
            route_list += [polyline.decode(route)]
        else:
            route_list += [()]
    test['len_route'] = [len(x) for x in route_list]
    # И посчитаем максимульную для между точками в каждом маршруте
    dist_route_list = []
    for route in route_list:
        dist_route_list += [[dist(x_0, x_1) for x_0, x_1 in zip(route[:-1], route[1:])]]
    test['max_route_part'] = [max(x) if len(x) > 0 else -1 for x in dist_route_list]
    # предсказываем ответ
    answer = np.clip(bst.predict(test[train_cols].values), 0, 2) * test['ETA'] - test['ETA'] * best_k
    return answer

## Проверим скорость предикта

In [99]:
%%time
test_val = test.loc[[0], :].copy()
for x in range(1000):
    answer = predict(test_val, bst, best_k, global_mean, mean_dict, cnt_dict)

Wall time: 16.8 s


In [100]:
print('TIME FOR 1 PREDICT: 16.8 ms' )

TIME FOR 1 PREDICT: 16.8 ms
